## Datum und Zeit in Rust

Wie arbeitet man in Rust mit Datum und Zeit?
- Es gibt ein Modul in der Standardbibliothek: [std::time](https://doc.rust-lang.org/std/time/index.html).
- Eine Websuche führt einem zum [Crate `chrono`](https://docs.rs/chrono/0.4.19/chrono/). 
> It aims to be a feature-complete superset of the time library. 
  - Für das Einlesen und Auslesen von Datum und Zeit hat `chrono` das [Modul `chrono::format`](https://docs.rs/chrono/0.4.19/chrono/format/index.html).
  - Diese Modul verwendet einen [String-Formatierer](https://docs.rs/chrono/0.4.19/chrono/format/strftime/index.html#specifiers).
  - Für die meisten Fälle sollen [DateTime::format](https://docs.rs/chrono/0.4.19/chrono/struct.DateTime.html#method.format) und [DateTime::parse_from_str](https://docs.rs/chrono/0.4.19/chrono/struct.DateTime.html#method.parse_from_str) ausreichen.
- Da ich mit Quartalen arbeiten möchte, fand ich in diesem [Webpost](https://stackoverflow.com/questions/62947343/nth-day-of-every-mth-month-in-rust) einen Hinweis auf [Crate (Kiste) `kronos`](https://crates.io/crates/kronos), der wiederum von `chrono` abhängt. Im Vergleich zu `chrono` ist die 
> Kronos is a tool for calculating date/times.
  - Interessant sind die Aufzählung der Datum/Zeit Einteilung [kronos::Grain](https://docs.rs/kronos/0.1.5/kronos/enum.Grain.html)
    - Second, Minute, Hour, 
    - Day, Week, Month,
    - Quarter, Half, Year,
    - Lustrum, Decade, Century, Millenium,
  - als auch die Aufzählung der Jahreszeiten [kronos::Season](https://docs.rs/kronos/0.1.5/kronos/enum.Season.html)
    - Spring, Summer, Autumn, Winter,
  - Allerdings sind die Elemente der Bibliothek wie Strukturen, Traits, und Funktionen undokumentiert. Sie dient wohl privaten Zwecken und vorallem dem generieren von Datum und Zeit Einträgen.
- Das Suchen von "date month quarter" führt einen zur [Crate (Kiste) `date_calculations`](https://crates.io/crates/date-calculations), die eine Reihe von Funktionen bereitstellt, um von einem Datum aus den Anfang, das Ende, das vorherige, oder das nächste Jahr (Monat, Quartal, Woche) zu berechnen.


### std::time

In [2]:
let now = std::time::SystemTime::now();

// we sleep for 2 seconds
std::thread::sleep(std::time::Duration::new(2, 0));
// it prints '2'
match now.elapsed() { 
    // SystemTime::elapsed(&self) -> Result<Duration, SystemTimeError>
    Ok(duration) => { println!("{}", duration.as_secs()); }
    Err(error) => { panic!("Error: {:?}", error); }
}

2


()

In [3]:
match std::time::SystemTime::now().duration_since(std::time::SystemTime::UNIX_EPOCH) {
    Ok(n) => println!("1970-01-01 00:00:00 UTC was {} seconds ago!", n.as_secs()),
    Err(_) => panic!("SystemTime before UNIX EPOCH!"),
}

1970-01-01 00:00:00 UTC was 1672978251 seconds ago!


()

### chrono

In [4]:
:dep chrono = {version = "0.4"}
// Source: https://rust-lang-nursery.github.io/rust-cookbook/datetime.html

let utc: chrono::DateTime<chrono::Utc> = chrono::Utc::now();
println!("UTC time now: {}", utc);

let rfc2822 = chrono::DateTime::parse_from_rfc2822("Tue, 1 Jul 2003 10:52:37 +0200")?;
println!("{}", rfc2822);

let rfc3339 = chrono::DateTime::parse_from_rfc3339("1996-12-19T16:39:57-08:00")?;
println!("{}", rfc3339);

let custom = chrono::DateTime::parse_from_str("5.8.1994 8:00 am +0000", "%d.%m.%Y %H:%M %P %z")?;
println!("{}", custom);

let time_only = chrono::NaiveTime::parse_from_str("23:56:04", "%H:%M:%S")?;
println!("{}", time_only);

let date_only = chrono::NaiveDate::parse_from_str("2015-09-05", "%Y-%m-%d")?;
println!("{}", date_only);

let no_timezone = chrono::NaiveDateTime::parse_from_str("2015-09-05 23:56:04", "%Y-%m-%d %H:%M:%S")?;
println!("{}", no_timezone);

let local: chrono::DateTime<chrono::Local> = chrono::Local::now();
println!("LOCAL time now: {}", local)

UTC time now: 2023-01-06 04:11:38.380163874 UTC
2003-07-01 10:52:37 +02:00
1996-12-19 16:39:57 -08:00
1994-08-05 08:00:00 +00:00
23:56:04
2015-09-05
2015-09-05 23:56:04
LOCAL time now: 2023-01-06 05:11:38.380213524 +01:00


()

### kronos

In [5]:
:dep chrono = {version = "0.4"}
:dep kronos = {version = "0.1.5"}

// Reference time: Tuesday, 5th Feb 2019
let date_time = chrono::NaiveDate::from_ymd(2019, 2, 5).and_hms(0, 0, 0);
println!("{}", date_time);

// fn future(&self, t0: &NaiveDateTime) -> Box<dyn Iterator<Item = Range>>
// fn past(&self, t0: &NaiveDateTime) -> Box<dyn Iterator<Item = Range>>

// First Monday after a date reference time
println!("{:?}", kronos::TimeSequence::future(
    &kronos::Weekday(1), 
    &date_time)
    .next()
    .unwrap());

println!("{:?}", kronos::TimeSequence::future(
    &kronos::Grains(kronos::Grain::Quarter), 
    &date_time)
    .next()
    .unwrap());
println!("{:?}", kronos::TimeSequence::past(
    &kronos::Grains(kronos::Grain::Quarter), 
    &date_time)
    .next()
    .unwrap());

// kronos has no documentation, looks as if BOOL stands for north(true) and south(false) half of earth
println!("{:?}", kronos::TimeSequence::future(
    &kronos::Seasons(kronos::Season::Spring, true), 
    &date_time)
    .next()
    .unwrap());
println!("{:?}", kronos::TimeSequence::past(
    &kronos::Seasons(kronos::Season::Spring, true), 
    &date_time)
    .next()
    .unwrap());


2019-02-05 00:00:00
Range { start: 2019-02-11T00:00:00, end: 2019-02-12T00:00:00, grain: Day }
Range { start: 2019-01-01T00:00:00, end: 2019-04-01T00:00:00, grain: Quarter }
Range { start: 2018-10-01T00:00:00, end: 2019-01-01T00:00:00, grain: Quarter }
Range { start: 2019-03-21T00:00:00, end: 2019-06-21T00:00:00, grain: Day }
Range { start: 2018-03-21T00:00:00, end: 2018-06-21T00:00:00, grain: Day }


In [6]:
:dep chrono = {version = "0.4"}
:dep kronos = {version = "0.1.5"}

kronos::TimeSequence::past(
    &kronos::LastOf(2, kronos::Weekend, kronos::Grains(kronos::Grain::Year)),
    &chrono::NaiveDateTime::from_timestamp(chrono::Local::now().timestamp(),0),
)
.next()
.unwrap()
/* 
   Following for-loop panics thanks to deadend in kronos

for range_item in kronos::TimeSequence::past(
...
)
{
    println!("{:?}", range_item)
}

thread '<unnamed>' panicked at 'assertion failed: i <= INFINITE_FUSE', /home/tilo/.cargo/registry/src/github.com-1ecc6299db9ec823/kronos-0.1.5/src/seq_lastof.rs:42:39
*/

Range { start: 2022-12-24T00:00:00, end: 2022-12-26T00:00:00, grain: Day }

In [7]:
:dep chrono = {version = "0.4"}
:dep kronos = {version = "0.1.5"}

kronos::TimeSequence::past(
    &kronos::Grains(kronos::Grain::Quarter),
    &chrono::NaiveDateTime::from_timestamp(chrono::Local::now().timestamp(),0),
)
.next()
.unwrap()
.duration()

Duration { secs: 7948800, nanos: 0 }

### date-calculations

In [8]:
:dep chrono = {version = "0.4"}
:dep date-calculations = {version = "0.1.1"}

let twenty_twentyone : chrono::NaiveDate = chrono::NaiveDate::from_ymd(2021, 1, 31);

assert_eq!(
    chrono::Datelike::year(&date_calculations::next_year(&twenty_twentyone).unwrap()), 
    2022);
assert_eq!(
    chrono::Datelike::month(&date_calculations::next_year(&twenty_twentyone).unwrap()), 
    1);
assert_eq!(
    chrono::Datelike::day(&date_calculations::next_year(&twenty_twentyone).unwrap()), 
    1);

assert_eq!(
    chrono::Datelike::year(&date_calculations::previous_quarter(&twenty_twentyone).unwrap()), 
    2020);
assert_eq!(
    chrono::Datelike::month(&date_calculations::previous_quarter(&twenty_twentyone).unwrap()), 
    10);
assert_eq!(
    chrono::Datelike::day(&date_calculations::previous_quarter(&twenty_twentyone).unwrap()), 
    1);


## Wie funktioniert die `chrono`-Kiste

[Kiste `chrono`](https://crates.io/crates/chrono) möchte eine Übermenge der - vermeintlich - eingestellten Bibliothek [`time`](https://docs.rs/time/latest/time/index.html) sein. Auch wenn `chrono` in seiner Dokumentation auf [rust-lang-deprecated/time](https://github.com/rust-lang-deprecated/time) zeigt, so wird der Link heute (2022-05-10) auf die Organisation [`time-rs`]() weitergeleitet. Egal.

`chrono` hat je drei Grundlegende Typen mit und ohne Zeitzonen Unterstützung:
- [`Date`](https://docs.rs/chrono/0.4/chrono/struct.Date.html), (non-existent) `Time`, [`DateTime`](https://docs.rs/chrono/0.4/chrono/struct.DateTime.html) parametrisiert mit Zeitzonen-Offset [`Utc`](https://docs.rs/chrono/0.4/chrono/offset/struct.Utc.html), [`local`](https://docs.rs/chrono/0.4/chrono/offset/struct.Local.html), oder [`Fixed`](https://docs.rs/chrono/0.4/chrono/offset/struct.FixedOffset.html) (Zeitzone mit fixed offset, von UTC-23:59:59 bis UTC+23:59:59).
- [`NaiveDate`](https://docs.rs/chrono/0.4/chrono/naive/struct.NaiveDate.html), [`NaiveTime`](https://docs.rs/chrono/0.4/chrono/naive/struct.NaiveTime.html), [`NaiveDateTime`](https://docs.rs/chrono/0.4/chrono/naive/struct.NaiveDateTime.html)

Auf die Eigenschafter der grundlegenden Typen greift man über die beiden Traits [`Datelike`](https://docs.rs/chrono/0.4/chrono/trait.Datelike.html) und [`Timelike`](https://docs.rs/chrono/0.4/chrono/trait.Timelike.html) zu.

In [9]:
:dep chrono = {version = "0.4"}

In [10]:
let utc: chrono::DateTime<chrono::Utc> = chrono::Utc::now();
let local: chrono::DateTime<chrono::Local> = chrono::Local::now();

println!("Universal time: {},", utc);
println!("Local     time: {},", local);


Universal time: 2023-01-06 04:16:45.657303114 UTC,
Local     time: 2023-01-06 05:16:45.657304034 +01:00,


In [11]:
chrono::NaiveDate::parse_from_str(format!("{}-01","2022-02").as_str(), "%Y-%m-%d")?


2022-02-01

In [12]:
chrono::NaiveDate::parse_from_str("2022-04-12", "%Y-%m-%d")?


2022-04-12

Using the [`TimeZone`](https://docs.rs/chrono/0.4.19/chrono/offset/trait.TimeZone.html) methods on a `FixedOffset` struct is the preferred way to construct `DateTime<FixedOffset>` instances. 

In [13]:
let hour = 3600;
let dt: chrono::DateTime<chrono::FixedOffset> = 
        chrono::TimeZone::ymd(&chrono::FixedOffset::east(5 * hour), 2016, 11, 08)
        .and_hms(0, 0, 0);

assert_eq!(&dt.to_rfc3339(), "2016-11-08T00:00:00+05:00");

// property accessors
use chrono::Datelike;
assert_eq!((dt.year(), dt.month(), dt.day()), (2016, 11, 08));
assert_eq!((dt.month0(), dt.day0()), (10, 7)); // for unfortunate souls
assert_eq!(dt.weekday(), chrono::Weekday::Tue);
assert_eq!(dt.weekday().number_from_monday(), 2); // Mon=1, ..., Sun=7
assert_eq!(dt.ordinal(), 313); // the day of year
assert_eq!(dt.num_days_from_ce(), 736276); // the number of days from and including Jan 1, 1

// time zone accessor
assert_eq!(dt.timezone(), chrono::FixedOffset::east(5 * 3600));


In [14]:
assert_eq!(chrono::NaiveDate::from_ymd(2015, 9, 8).year(), 2015);
assert_eq!(chrono::NaiveDate::from_ymd(-308, 3, 14).year(), -308); // 309 BCE


## Julia Abfrage-Funktionen mit der `chrono`-Kiste

Nachbau der Query-Funktionen und notwendige Teile der Accessor-Funktionen der Julia Bibliothek [Dates](https://docs.julialang.org/en/v1/stdlib/Dates), um aus einem Chronos::Datelike Objekt das Jahr, das Quartal, den Monat, den Tag abzuleiten.

Die Julia Implementationen verwenden verschiedene mathematische Operationen für Division:
- [div()](https://docs.julialang.org/en/v1/base/math/#Base.div)
  `div(a, b) = div(a, b, RoundToZero)`
- [fld()](https://docs.julialang.org/en/v1/base/math/#Base.fld)
  `fld(a, b) = div(a, b, RoundDown)`
- [cld()](https://docs.julialang.org/en/v1/base/math/#Base.cld)
  `cld(a, b) = div(a, b, RoundUp)`
- [rem()](https://docs.julialang.org/en/v1/base/math/#Base.rem)

Rust kennt die Operatoren 
- [std::ops::Div](https://doc.rust-lang.org/std/ops/trait.Div.html), The division operator `\`.
- und [std::ops::Rem](https://doc.rust-lang.org/std/ops/trait.Rem.html), The remainder operator `%`.

Seit einiger Zeit kennt Rust nun auch folgende Operatoren als Methoden der Integer Typen:
- `div_ceil` (nightly-only experimental API. ([int_roundings #88581](https://github.com/rust-lang/rust/issues/88581)))
- `div_euclid` (1.38.0 (const: 1.52.0))
- `div_floor` (nightly-only experimental API. ([int_roundings #88581](https://github.com/rust-lang/rust/issues/88581)))
- `rem_euclid` (1.38.0 (const: 1.52.0)) [RFC 2169](https://github.com/rust-lang/rfcs/blob/master/text/2169-euclidean-modulo.md)


In [32]:
assert_eq!(9i32.div_euclid(4), 2);

In [36]:
i32::rem_euclid(9i32, 4i32)

1

In [15]:
fn isleapyear_num(y: i32) -> bool {
    ((y % 4) == 0) && (
    ((y % 100) != 0) || 
    ((y % 400) == 0))
}

isleapyear_num(2004)


true

Rust [keyword `where`](https://doc.rust-lang.org/std/keyword.where.html): Add constraints that must be upheld to use an item.

In [16]:
fn isleapyear<D>(dt: D) -> bool where D: chrono::Datelike {
    isleapyear_num(dt.year())
}

isleapyear(chrono::NaiveDate::from_ymd(2015, 9, 8))


false

Rust [keyword `const`](https://doc.rust-lang.org/std/keyword.const.html): Compile-time constants, compile-time evaluable functions, and raw pointers.

In [17]:
fn daysinmonth_num(y: i32, m: u32) -> u32 {
    assert!((1 <= m) && (m <= 12));
    const DAYSINMONTH: [u32;12] = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31];
    let month_idx: usize = <u32 as std::convert::TryInto<usize>>::try_into(m).unwrap();
    let leap_ofs: u32 = <bool as std::convert::TryInto<u32>>::try_into(
        (m == 2) && 
        isleapyear_num(y)).unwrap();
    DAYSINMONTH[month_idx - 1] + leap_ofs
}

daysinmonth_num(2004, 2)


29

In [18]:
fn daysinmonth<D>(dt: D) -> u32 where D: chrono::Datelike {
    daysinmonth_num(dt.year(), dt.month())
}

daysinmonth(chrono::TimeZone::ymd(&chrono::Utc, 2004, 2, 25))


29

In [ ]:
fn month_num(d: u32) -> u32 {
}
